Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

* engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
* month: an integer representing the month for which rental data is to be retrieved.
* year: an integer representing the year for which rental data is to be retrieved.

The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.



In [9]:
import mysql.connector
import getpass
password = getpass.getpass()
import pandas as pd


engine = mysql.connector.connect(
    host='localhost',
    user='root',
    password=password,
    database='sakila'
)

In [15]:
def rental_months(engine, month, year):
    cursor = engine.cursor()

    query = """
    SELECT *,
           DATE_FORMAT(rental_date, '%m') as rental_month,
           DATE_FORMAT(rental_date, '%Y') as rental_year
    FROM rental
    WHERE DATE_FORMAT(rental_date, '%m') = %s AND DATE_FORMAT(rental_date, '%Y') = %s
    """
    
    cursor.execute(query, (month, year))
    query_1 = cursor.fetchall()

    data = []

    for row in query_1:
        data.append(row)

    cursor.close()

    df = pd.DataFrame(data, columns=[
        "rental_id", "rental_date", "inventory_id", "customer_id", 
        "return_date", "staff_id", "last_update", "rental_month", "rental_year"
    ])
    return df

In [17]:
password = getpass.getpass()
engine = mysql.connector.connect(
    host='localhost',
    user='root',
    password=password,
    database='sakila'
)

df = rental_months(engine, '05', '2005')
engine.close()

df.head(10)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update,rental_month,rental_year
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53,05,2005
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53,05,2005
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53,05,2005
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53,05,2005
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53,05,2005
5,6,2005-05-24 23:08:07,2792,549,2005-05-27 01:32:07,1,2006-02-15 21:30:53,05,2005
6,7,2005-05-24 23:11:53,3995,269,2005-05-29 20:34:53,2,2006-02-15 21:30:53,05,2005
7,8,2005-05-24 23:31:46,2346,239,2005-05-27 23:33:46,2,2006-02-15 21:30:53,05,2005
8,9,2005-05-25 00:00:40,2580,126,2005-05-28 00:22:40,1,2006-02-15 21:30:53,05,2005
9,10,2005-05-25 00:02:21,1824,399,2005-05-31 22:44:21,2,2006-02-15 21:30:53,05,2005
